In [8]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime

In [9]:
load_dotenv('../.env')

True

In [10]:
TASK_NAME = "relevance_check_v0"

def get_run_id():
    return os.getenv('RUNID')

RUNID = get_run_id()

INPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHFILES"

RUN_TIME = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')

OUTPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHID"

os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)

print(f"Run ID: {RUNID} at {RUN_TIME}")

Run ID: RUNID_2 at 2025-05-30 17:50:07


In [11]:
AZURE_OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT=os.getenv('AZURE_OPENAI_ENDPOINT')
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [12]:
def get_batchfiles():
    batchfiles = []
    for filename in os.listdir(INPUT_DATA_PATH):
        if filename.endswith('.jsonl'):
            b = {
                'path': os.path.join(INPUT_DATA_PATH, filename),
                'id' : filename.split('.')[0].split('_')[-1],
            }
            batchfiles.append(b)
    return sorted(batchfiles)

batchfiles = get_batchfiles()

In [13]:
def save_batchid(i, batch_id):
    batchid_filename = f"{RUNID}--{TASK_NAME}_BATCHID_{i}.txt"
    with open(os.path.join(OUTPUT_DATA_PATH, batchid_filename), "w") as f:
        f.write(batch_id)

In [14]:

for bathcfile in batchfiles:

  i = bathcfile['id']
  path = bathcfile['path']

  print("Submitting {}".format(path))

  file = client.files.create(
    file=open(path, "rb"),
    purpose="batch"
  )

  print(file.model_dump_json(indent=2))
  file_id = file.id

  # Submit a batch job with the file
  batch_response = client.batches.create(
      input_file_id=file_id,
      endpoint="/chat/completions",
      completion_window="24h",
  )

  # Save batch ID for later use
  batch_id = batch_response.id

  print(batch_response.model_dump_json(indent=2)) 

  save_batchid(i, batch_id)
  print(f"Batch ID for file {i}: {batch_id}")
  print("Batch job submitted successfully.")
  print("  ")

Submitting ../local_tests_data/azure_openai_batch_processing_files/RUNID_2/relevance_check_v0/BATCHFILES/RUNID_2--relevance_check_v0_BATCHFILE_0.jsonl
{
  "id": "file-3b10b5d91762403bbca252db1c4779c3",
  "bytes": 44298,
  "created_at": 1748620209,
  "filename": "RUNID_2--relevance_check_v0_BATCHFILE_0.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
{
  "id": "batch_83596626-284c-4d57-a65b-ea6fd8063094",
  "completion_window": "24h",
  "created_at": 1748620213,
  "endpoint": "/chat/completions",
  "input_file_id": "file-3b10b5d91762403bbca252db1c4779c3",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1748706610,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed"